In [ ]:
import pandas as pd
!pip install haversine
import numpy as np
import folium
from sklearn.cluster import KMeans
from geopy.distance import geodesic
import random
from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import AgglomerativeClustering
from haversine import haversine
from datetime import datetime
from scipy.spatial.distance import cdist

# Load Excel File (Upload in Colab first)
file_path = "/content/SmartRoute Optimizer.xlsx"

# Read Data
shipments_df = pd.read_excel(file_path, sheet_name="Shipments_Data").head(400)
vehicles_df = pd.read_excel(file_path, sheet_name="Vehicle_Information")
store_df = pd.read_excel(file_path, sheet_name="Store Location")


# Function to convert time to minutes from midnight
def time_to_minutes(timeslot):
    start_time, end_time = timeslot.split('-')
    start_time = datetime.strptime(start_time, "%H:%M:%S")
    end_time = datetime.strptime(end_time, "%H:%M:%S")

    # Convert times to minutes from midnight
    start_minutes = start_time.hour * 60 + start_time.minute
    end_minutes = end_time.hour * 60 + end_time.minute

    return start_minutes, end_minutes

# Apply time conversion to the DataFrame
shipments_df['TimeSlot'] = shipments_df['Delivery Timeslot'].apply(time_to_minutes)

# Normalize time so that the earliest start time is 0
min_start_time = min(shipments_df['TimeSlot'].apply(lambda x: x[0]))
shipments_df['TimeSlot'] = shipments_df['TimeSlot'].apply(lambda x: (x[0] - min_start_time, x[1] - min_start_time))

# Create a global distance matrix for geographical distances
def create_geo_distance_matrix(shipments):
    num_shipments = len(shipments)
    geo_dist_matrix = np.zeros((num_shipments, num_shipments))

    for i in range(num_shipments):
        for j in range(num_shipments):
            if i == j:
                continue
            geo_dist_matrix[i][j] = haversine((shipments.iloc[i]['Latitude'], shipments.iloc[i]['Longitude']),
                                              (shipments.iloc[j]['Latitude'], shipments.iloc[j]['Longitude']))
    return geo_dist_matrix

# Create the global geographical distance matrix
geo_dist_matrix = create_geo_distance_matrix(shipments_df)

# Function to adjust the global distance matrix with time component
def adjusted_distance_matrix(geo_dist_matrix, shipments, alpha=10):
    num_shipments = len(shipments)
    dist_matrix = np.zeros((num_shipments, num_shipments))

    for i in range(num_shipments):
        for j in range(num_shipments):
            if i == j:
                continue
            # Calculate the time difference based on the normalized start times
            time_dist = abs(shipments.iloc[i]['TimeSlot'][0] - shipments.iloc[j]['TimeSlot'][0])
            dist_matrix[i][j] = geo_dist_matrix[i][j] + alpha * time_dist

    return dist_matrix

# Clustering Shipments using Agglomerative Clustering
def cluster_shipments(shipments, geo_dist_matrix, num_clusters=5, alpha=10):
    distance_matrix = adjusted_distance_matrix(geo_dist_matrix, shipments, alpha)
    clustering = AgglomerativeClustering(n_clusters=num_clusters, metric='precomputed', linkage='complete')
    shipments['Cluster'] = clustering.fit_predict(distance_matrix)
    return shipments

# Apply clustering with different alpha values
# alpha_values = [5, 10, 15]  # Example alpha values
# for alpha in alpha_values:
shipments_df = cluster_shipments(shipments_df, geo_dist_matrix, num_clusters=5, alpha=100000)
    # print(f"Clustering with alpha = {alpha}:")
    # print(shipments_df)

In [ ]:
from geopy.distance import geodesic
import pandas as pd

def precompute_distances(shipments, store_location):
    """
    Precompute distances between the store and all shipments, and between all pairs of shipments.

    Parameters:
        shipments (pd.DataFrame): DataFrame containing Shipment_ID, Latitude, Longitude.
        store_location (pd.DataFrame): DataFrame containing the store's Latitude and Longitude.

    Returns:
        dict: A dictionary where keys are tuples of (location1, location2) and values are distances in km.
    """
    # Extract store coordinates
    store_lat, store_lon = store_location.iloc[0]["Latitute"], store_location.iloc[0]["Longitude"]
    store_coords = (store_lat, store_lon)

    # Initialize distance matrix
    distance_matrix = {}

    # Add store to shipments for distance calculation
    locations = [("Store", store_coords)] + list(zip(shipments["Shipment ID"], zip(shipments["Latitude"], shipments["Longitude"])))

    # Precompute distances
    for (loc1_name, loc1_coords) in locations:
        for (loc2_name, loc2_coords) in locations:
            if (loc1_name, loc2_name) not in distance_matrix:
                distance = geodesic(loc1_coords, loc2_coords).km
                distance_matrix[(loc1_name, loc2_name)] = distance
                distance_matrix[(loc2_name, loc1_name)] = distance  # Distance is symmetric

    return distance_matrix

def nearest_neighbor_heuristic(shipments, vehicles, store_location):
    """
    Nearest Neighbor Heuristic for assigning shipments to vehicles.
    Each trip starts and ends at the store, and the store is included in the route.

    Parameters:
        shipments (pd.DataFrame): DataFrame containing Shipment_ID, Latitude, Longitude, Delivery Timeslot, and Cluster.
        vehicles (pd.DataFrame): DataFrame containing Vehicle_Type, Number, Shipments_Capacity, and Max Trip Radius.
        store_location (pd.DataFrame): DataFrame containing the store's Latitude and Longitude.

    Returns:
        pd.DataFrame: DataFrame with assigned trips, including Trip_ID, Shipment_ID, Vehicle_Type, and Sequence.
    """
    # Precompute distances
    distance_matrix = precompute_distances(shipments, store_location)

    # Initialize variables
    trips = []  # To store the final trips
    shipment_ids = shipments["Shipment ID"].tolist()  # List of all shipment IDs
    assigned_shipments = set()  # To track assigned shipments

    # Iterate over each vehicle type
    for _, vehicle in vehicles.iterrows():
        vehicle_type = vehicle["Vehicle Type"]
        capacity = vehicle["Shipments_Capacity"]
        max_radius = vehicle["Max Trip Radius (in KM)"]

        # Assign shipments to the current vehicle
        for _ in range(int(vehicle["Number"])):  # Number of vehicles of this type
            current_location = "Store"  # Start at the store
            trip_shipments = []  # Shipments assigned to this trip
            trip_distance = 0  # Total distance for this trip

            while len(trip_shipments) < capacity:
                # Find the nearest unassigned shipment
                nearest_shipment = None
                nearest_distance = float("inf")

                for shipment_id in shipment_ids:
                    if shipment_id not in assigned_shipments:
                        distance = distance_matrix[(current_location, shipment_id)]

                        # Check if the shipment is within the vehicle's max radius
                        if distance < nearest_distance and (trip_distance + distance) <= max_radius:
                            nearest_shipment = shipment_id
                            nearest_distance = distance

                # If no more shipments can be assigned, break
                if nearest_shipment is None:
                    break

                # Assign the nearest shipment to the trip
                trip_shipments.append(nearest_shipment)
                assigned_shipments.add(nearest_shipment)

                # Update the trip distance
                trip_distance += nearest_distance
                current_location = nearest_shipment

            # Add the return distance to the store
            if trip_shipments:
                return_distance = distance_matrix[(current_location, "Store")]
                trip_distance += return_distance

                # Add the store at the start and end of the trip
                trip_shipments = ["Store"] + trip_shipments + ["Store"]

                # Add the trip to the list of trips
                trips.append({
                    "Trip_ID": len(trips) + 1,
                    "Vehicle_Type": vehicle_type,
                    "Shipments": trip_shipments,
                    "Total_Distance": trip_distance
                })

    # Convert the trips list to a DataFrame
    trips_df = pd.DataFrame(trips)
    return trips_df


# Run the Nearest Neighbor Heuristic
trips_df = nearest_neighbor_heuristic(shipments_df, vehicles_df, store_df)

print("Optimized Trips (with Store at Start and End):")
print(trips_df)

Optimized Trips (with Store at Start and End):
     Trip_ID Vehicle_Type                                          Shipments  \
0          1           3W           [Store, 639, 493, 1054, 892, 269, Store]   
1          2           3W            [Store, 721, 398, 680, 478, 470, Store]   
2          3           3W           [Store, 517, 384, 914, 1007, 935, Store]   
3          4           3W             [Store, 882, 58, 410, 886, 859, Store]   
4          5           3W           [Store, 187, 1245, 464, 307, 358, Store]   
..       ...          ...                                                ...   
101      102           4W  [Store, 664, 460, 223, 427, 169, 128, 217, 629...   
102      103           4W  [Store, 153, 245, 429, 674, 1098, 1074, 1089, ...   
103      104           4W  [Store, 254, 349, 409, 867, 607, 98, 827, 761,...   
104      105           4W  [Store, 584, 787, 208, 762, 605, 734, 906, 135...   
105      106           4W  [Store, 1174, 222, 12, 1002, 585, 103, 773, 58

In [ ]:
from itertools import permutations

def tsp_optimization(trip_shipments, shipments, store_location):
    """
    Optimize the sequence of shipments in a trip using TSP.
    The trip starts and ends at the store location.

    Parameters:
        trip_shipments (list): List of shipment IDs in the trip.
        shipments (pd.DataFrame): DataFrame containing all shipments.
        store_location (pd.DataFrame): DataFrame containing the store's Latitude and Longitude.

    Returns:
        list: Optimized sequence of shipment IDs (including the store at the start and end).
        float: Total distance of the optimized trip.
    """
    # Extract store coordinates
    store_lat, store_lon = store_location.iloc[0]["Latitute"], store_location.iloc[0]["Longitude"]

    # Get coordinates of all shipments in the trip
    shipment_coords = []
    for shipment_id in trip_shipments:
        if shipment_id == "Store":
            continue  # Skip the store
        shipment = shipments[shipments["Shipment ID"] == shipment_id].iloc[0]
        shipment_coords.append((shipment["Latitude"], shipment["Longitude"]))

    # Generate all possible permutations of the shipment sequence
    min_distance = float("inf")
    best_sequence = None

    for sequence in permutations(range(len(shipment_coords))):
        current_distance = 0
        current_location = (store_lat, store_lon)  # Start at the store

        # Calculate the total distance for this sequence
        for idx in sequence:
            next_location = shipment_coords[idx]
            current_distance += geodesic(current_location, next_location).km
            current_location = next_location

        # Return to the store
        current_distance += geodesic(current_location, (store_lat, store_lon)).km

        # Update the best sequence if this one is better
        if current_distance < min_distance:
            min_distance = current_distance
            best_sequence = [trip_shipments[idx + 1] for idx in sequence]  # +1 to skip the store

    # Add the store at the start and end of the sequence
    best_sequence = ["Store"] + best_sequence + ["Store"]

    return best_sequence, min_distance

In [ ]:
# Optimize each trip using TSP
optimized_trips = []
for _, trip in trips_df.iterrows():
    if len(trip["Shipments"]) <= 9:  # Subtract 2 to exclude the "Store" at start and end
        # Apply TSP optimization for small trips
        optimized_sequence, optimized_distance = tsp_optimization(trip["Shipments"], shipments_df, store_df)
    else:
        # Keep the original route for large trips
        optimized_sequence = trip["Shipments"]
        optimized_distance = trip["Total_Distance"]
    optimized_trips.append({
        "Trip_ID": trip["Trip_ID"],
        "Vehicle_Type": trip["Vehicle_Type"],
        "Shipments": optimized_sequence,
        "Total_Distance": optimized_distance
    })

# Convert the optimized trips to a DataFrame
optimized_trips_df = pd.DataFrame(optimized_trips)

print("Optimized Trips (Local Minima):")
print(optimized_trips_df)

Optimized Trips (Local Minima):
     Trip_ID Vehicle_Type                                          Shipments  \
0          1           3W           [Store, 639, 493, 1054, 269, 892, Store]   
1          2           3W            [Store, 470, 478, 680, 398, 721, Store]   
2          3           3W           [Store, 517, 384, 914, 935, 1007, Store]   
3          4           3W             [Store, 882, 58, 410, 886, 859, Store]   
4          5           3W           [Store, 187, 1245, 464, 307, 358, Store]   
..       ...          ...                                                ...   
101      102           4W  [Store, 664, 460, 223, 427, 169, 128, 217, 629...   
102      103           4W  [Store, 153, 245, 429, 674, 1098, 1074, 1089, ...   
103      104           4W  [Store, 254, 349, 409, 867, 607, 98, 827, 761,...   
104      105           4W  [Store, 584, 787, 208, 762, 605, 734, 906, 135...   
105      106           4W  [Store, 1174, 222, 12, 1002, 585, 103, 773, 58...   

     To

In [ ]:
import pandas as pd


# Create a mapping between shipment IDs and their time slots
shipment_time_slot_mapping = dict(zip(shipments_df["Shipment ID"], shipments_df["Delivery Timeslot"]))

print(len(shipment_time_slot_mapping))
# Access a specific key from the dictionary
# key_to_access = 1  # Replace with the key you want to access
# Create the final output DataFrame
final_output = []

for _, trip in trips_df.iterrows():
    trip_id = trip["Trip_ID"]
    vehicle_type = trip["Vehicle_Type"]
    shipments = trip["Shipments"]
    total_distance = trip["Total_Distance"]

    # Get vehicle capacity and max trip radius
    vehicle_capacity = vehicles_df[vehicles_df["Vehicle Type"] == vehicle_type]["Shipments_Capacity"].values[0]
    max_trip_radius = vehicles_df[vehicles_df["Vehicle Type"] == vehicle_type]["Max Trip Radius (in KM)"].values[0]

    # Calculate the number of shipments (excluding the store)
    num_shipments = len([s for s in shipments if s != "Store"])

    # Calculate CAPACITY_UTI, TIME_UTI, and COV_UTI
    capacity_uti = num_shipments / vehicle_capacity  # Capacity Utilization
    trip_time = total_distance * 5  # Assuming 5 minutes per km (adjust as needed)
    time_uti = trip_time / 180  # Assuming 3-hour time slot (180 minutes)

    # COV_UTI (Distance Utilization) = Maximum distance point / Max Trip Radius
    # Here, we assume the maximum distance point is the total distance of the trip
    # If you have a different definition for "maximum distance point," adjust accordingly
    cov_uti = total_distance / max_trip_radius

    # Format the route as "Shop -> Shipment1 -> Shipment2 -> ... -> Shop"
    route = " -> ".join([str(s) if s != "Store" else "Shop" for s in shipments])

    # Get the time slots for the shipments in this trip
    # Get the time slots for the shipments in this trip
    shipment_ids = [s for s in shipments if s != "Store"]

    # Convert shipment_ids to integers if necessary
    shipment_ids = [int(shipment_id) for shipment_id in shipment_ids]

# Access the time slots, handling missing keys
    # print("Keys in shipment_time_slot_mapping:", list(type(shipment_time_slot_mapping.keys()))[:5])  # First 5 keys
    time_slots = [shipment_time_slot_mapping.get(shipment_id, "Unknown Time Slot") for shipment_id in shipment_ids]
    # time_slots = [shipment_time_slot_mapping[shipment_id] for shipment_id in shipment_ids]

    # Add the trip to the final output
    final_output.append({
        "Vehicle Type": vehicle_type,
        "Total Shipments": num_shipments,
        "Shipments Delivered": ", ".join([str(s) for s in shipments if s != "Store"]),
        "Route": route,
        "MST Distance": round(total_distance, 2),
        "Trip Time": round(trip_time, 2),
        "Capacity Utilization": round(capacity_uti, 2),
        "Time Utilization": round(time_uti, 2),
        "COV_UTI (Distance Utilization)": round(cov_uti, 2),
        "Time Slots": ", ".join(time_slots)  # Add the time slots for the shipments
    })

# Convert the final output to a DataFrame
final_output_df = pd.DataFrame(final_output)


# Save the final output to an Excel file
final_output_df.to_excel("optimized_trips_final_output.xlsx", index=False)

# Display the final output
print("Final Output DataFrame:")
print(final_output_df)

400
Final Output DataFrame:
   Vehicle Type  Total Shipments                   Shipments Delivered  \
0            3W                5               398, 384, 187, 307, 358   
1            3W                5                 269, 157, 198, 374, 1   
2            3W                5                58, 410, 308, 163, 270   
3            3W                5               148, 400, 333, 104, 335   
4            3W                5                295, 347, 383, 171, 19   
..          ...              ...                                   ...   
66        4W-EV                8  357, 175, 178, 186, 273, 92, 290, 77   
67        4W-EV                8   59, 297, 402, 147, 33, 154, 136, 99   
68        4W-EV                4                     127, 249, 287, 78   
69        4W-EV                5                 222, 12, 103, 123, 14   
70        4W-EV                1                                   319   

                                                Route  MST Distance  \
0     Shop -

In [ ]:
from geopy.distance import geodesic

def nearest_neighbor_heuristic(shipments, vehicles, store_location):
    """
    Nearest Neighbor Heuristic for assigning shipments to vehicles.
    Each trip starts and ends at the store, and the store is included in the route.

    Parameters:
        shipments (pd.DataFrame): DataFrame containing Shipment_ID, Latitude, Longitude, Delivery Timeslot, and Cluster.
        vehicles (pd.DataFrame): DataFrame containing Vehicle_Type, Number, Shipments_Capacity, and Max Trip Radius.
        store_location (pd.DataFrame): DataFrame containing the store's Latitude and Longitude.

    Returns:
        pd.DataFrame: DataFrame with assigned trips, including Trip_ID, Shipment_ID, Vehicle_Type, and Sequence.
    """
    # Extract store coordinates
    store_lat, store_lon = store_location.iloc[0]["Latitute"], store_location.iloc[0]["Longitude"]

    # Initialize variables
    trips = []  # To store the final trips
    shipment_ids = shipments["Shipment ID"].tolist()  # List of all shipment IDs
    assigned_shipments = set()  # To track assigned shipments

    # Iterate over each vehicle type
    for _, vehicle in vehicles.iterrows():
        vehicle_type = vehicle["Vehicle Type"]
        capacity = vehicle["Shipments_Capacity"]
        max_radius = vehicle["Max Trip Radius (in KM)"]

        # Assign shipments to the current vehicle
        for _ in range(int(vehicle["Number"])):  # Number of vehicles of this type
            current_location = (store_lat, store_lon)  # Start at the store
            trip_shipments = []  # Shipments assigned to this trip
            trip_distance = 0  # Total distance for this trip

            while len(trip_shipments) < capacity:
                # Find the nearest unassigned shipment
                nearest_shipment = None
                nearest_distance = float("inf")

                for shipment_id in shipment_ids:
                    if shipment_id not in assigned_shipments:
                        shipment = shipments[shipments["Shipment ID"] == shipment_id].iloc[0]
                        shipment_location = (shipment["Latitude"], shipment["Longitude"])
                        distance = geodesic(current_location, shipment_location).km

                        # Check if the shipment is within the vehicle's max radius
                        if distance < nearest_distance and (trip_distance + distance) <= max_radius:
                            nearest_shipment = shipment
                            nearest_distance = distance

                # If no more shipments can be assigned, break
                if nearest_shipment is None:
                    break

                # Assign the nearest shipment to the trip
                trip_shipments.append(nearest_shipment["Shipment ID"])
                assigned_shipments.add(nearest_shipment["Shipment ID"])

                # Update the trip distance
                trip_distance += nearest_distance
                current_location = (nearest_shipment["Latitude"], nearest_shipment["Longitude"])

            # Add the return distance to the store
            if trip_shipments:
                return_distance = geodesic(current_location, (store_lat, store_lon)).km
                trip_distance += return_distance

                # Add the store at the start and end of the trip
                trip_shipments = ["Store"] + trip_shipments + ["Store"]

                # Add the trip to the list of trips
                trips.append({
                    "Trip_ID": len(trips) + 1,
                    "Vehicle_Type": vehicle_type,
                    "Shipments": trip_shipments,
                    "Total_Distance": trip_distance
                })

    # Convert the trips list to a DataFrame
    trips_df = pd.DataFrame(trips)
    return trips_df


# Run the Nearest Neighbor Heuristic
trips_df = nearest_neighbor_heuristic(shipments_df, vehicles_df, store_df)

print("Initial Trips (with Store at Start and End):")
print(trips_df)

In [ ]:
from geopy.distance import geodesic

def nearest_neighbor_heuristic(shipments, vehicles, store_location):
    """
    Nearest Neighbor Heuristic for assigning shipments to vehicles with timeslot constraints.
    Each trip starts and ends at the store, and the store is included in the route.

    Parameters:
        shipments (pd.DataFrame): DataFrame containing Shipment_ID, Latitude, Longitude, Delivery Timeslot, and Cluster.
        vehicles (pd.DataFrame): DataFrame containing Vehicle_Type, Number, Shipments_Capacity, and Max Trip Radius.
        store_location (pd.DataFrame): DataFrame containing the store's Latitude and Longitude.

    Returns:
        pd.DataFrame: DataFrame with assigned trips, including Trip_ID, Shipment_ID, Vehicle_Type, and Sequence.
    """
    # Extract store coordinates
    store_lat, store_lon = store_location.iloc[0]["Latitute"], store_location.iloc[0]["Longitude"]

    # Initialize variables
    trips = []  # To store the final trips
    shipment_ids = shipments["Shipment ID"].tolist()  # List of all shipment IDs
    assigned_shipments = set()  # To track assigned shipments

    # Iterate over each vehicle type
    for _, vehicle in vehicles.iterrows():
        vehicle_type = vehicle["Vehicle Type"]
        capacity = vehicle["Shipments_Capacity"]
        max_radius = vehicle["Max Trip Radius (in KM)"]

        # Assign shipments to the current vehicle
        for _ in range(int(vehicle["Number"])):  # Number of vehicles of this type
            current_location = (store_lat, store_lon)  # Start at the store
            trip_shipments = []  # Shipments assigned to this trip
            trip_distance = 0  # Total distance for this trip
            trip_timeslots = []  # To track timeslots of shipments in this trip

            while len(trip_shipments) < capacity:
                # Find the nearest unassigned shipment that fits the timeslot constraints
                nearest_shipment = None
                nearest_distance = float("inf")

                for shipment_id in shipment_ids:
                    if shipment_id not in assigned_shipments:
                        shipment = shipments[shipments["Shipment ID"] == shipment_id].iloc[0]
                        shipment_location = (shipment["Latitude"], shipment["Longitude"])
                        distance = geodesic(current_location, shipment_location).km

                        # Check if the shipment is within the vehicle's max radius
                        if distance < nearest_distance and (trip_distance + distance) <= max_radius:
                            # Check if the shipment's timeslot overlaps with existing shipments in the trip
                            shipment_timeslot = shipment["TimeSlot"]
                            overlaps = False
                            for existing_timeslot in trip_timeslots:
                                if (shipment_timeslot[0] < existing_timeslot[1] and
                                    shipment_timeslot[1] > existing_timeslot[0]):
                                    overlaps = True
                                    break

                            if not overlaps:
                                nearest_shipment = shipment
                                nearest_distance = distance

                # If no more shipments can be assigned, break
                if nearest_shipment is None:
                    break

                # Assign the nearest shipment to the trip
                trip_shipments.append(nearest_shipment["Shipment ID"])
                assigned_shipments.add(nearest_shipment["Shipment ID"])
                trip_timeslots.append(nearest_shipment["TimeSlot"])

                # Update the trip distance
                trip_distance += nearest_distance
                current_location = (nearest_shipment["Latitude"], nearest_shipment["Longitude"])

            # Add the return distance to the store
            if trip_shipments:
                return_distance = geodesic(current_location, (store_lat, store_lon)).km
                trip_distance += return_distance

                # Add the store at the start and end of the trip
                trip_shipments = ["Store"] + trip_shipments + ["Store"]

                # Add the trip to the list of trips
                trips.append({
                    "Trip_ID": len(trips) + 1,
                    "Vehicle_Type": vehicle_type,
                    "Shipments": trip_shipments,
                    "Total_Distance": trip_distance,
                    "Timeslots": trip_timeslots
                })

    # Convert the trips list to a DataFrame
    trips_df = pd.DataFrame(trips)
    return trips_df


# Run the Nearest Neighbor Heuristic with timeslot constraints
trips_df = nearest_neighbor_heuristic(shipments_df, vehicles_df, store_df)

print("Trips with Timeslot Constraints:")
print(trips_df)